# ORCHID Clinical Trial: statistical analysis reproduction

# Version 1.0

This notebook reproduces the statistical analysis of the ORCHID clinical trial. Results have been published to JAMA, on November 7th 2021: ["Effect of Hydroxychloroquine on Clinical Status at 14 Days in Hospitalized Patients With COVID-19"](https://jamanetwork.com/journals/jama/fullarticle/2772922). The statistical analysis plan can be found on [clinicaltrials.gov](https://clinicaltrials.gov/ct2/show/NCT04332991?term=orchid&cond=Covid19&cntry=US&draw=2&rank=1). 

The clinical trial has been conducted between April and July 2020, and stopped before enrollment completion for futility, finding no difference of efficacy between hydroxychloroquine and placebo. This notebook is a reproduction of the clinical trial results based on the clinical trial protocol and the investigators original source code.

## Requirements

*This notebook has been tested to work with R version 4.2.2 on 3/09/2023*. 

Below is the output of the sessionInfo() function:


In [ ]:
sessionInfo()

# Data Access using PIC-SURE API

User access authentication works through a security token, which is passed to the API using the token.txt file (file to be created by the user). In order to know how to get your security token, please see [`README.md` file](../README.md) and [`Workspace_setup.ipynb` file](./Workspace_setup.ipynb).

# ORCHID Clinical Trial

It is a multi-center, double blinded, randomized clinical trial conducted to assess the efficacy of hydroxychloroquine in the treatment of COVID-19 in hospitalized patients.

NHLBI made available the data to every authorized investigators. Hence, this notebook enables anybody with authorized credentials to reproduce the ORCHID clinical trial results by showing how to:
1. Access the data using the PIC-SURE API
2. Reproduce the results of this study using the open-source R programming languages

# Overview of the statistical analysis plan

The primary outcome is the COVID-19 Outcome Scale (COS) assess at 14 days. The scale comprises 7 levels, as follows:
- 1, Dead
- 2, Hospitalized on invasive mechanical ventilation or ECMO
- 3, Hospitalized on non-invasive ventilation or high flow nasal cannula
- 4, Hospitalized on supplemental oxygen
- 5, Hospitalized not on supplemental oxygen
- 6, Not hospitalized with limitation in activity (continued symptoms)
- 7, Not hospitalized without limitation in activity (no symptoms)

This scale will also be assessed at different timepoints as secondary outcomes: day-3, day-7, and day-28.

This scale will be treated as an ordered factor, and thus this outcome will be analyzed using a proportional odds regression model. 

Other secondary outcomes will be considered. Death, and composite of death and ECMO will be analyzed using logistic regression models. Time to recovery (patient without oxygen supplementation -- ie COS level 6 or 7) and time to discharge will be analyzed using survival models. Support free-days (hospital, oxygen, ICU, ventilation, and vasopressor) will be treated as ordered factors and analyzed using proportional odds regression models.

The multivariable model adjusts the treatment effect for the following 5 confoundings:
- Age at randomization
- Sex
- Clinical status as assessed by the COVID Ordinal Outcome Scale at randomization
- Sequential Organ Failure Assessment (SOFA) score at randomization
- Duration of acute respiratory infection symptoms prior to randomization

# Packages Installation 

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="main", force=T, quiet=FALSE)

In [ ]:
# If you are NOT using the Seven Bridges PIC-SURE image, you may need to install the following packages.
#install.packages('ggtext', repos='http://cran.rstudio.com/')
#install.packages('quantreg', repos='http://cran.rstudio.com/')
#install.packages('DescTools', repos='http://cran.rstudio.com/')
#install.packages('survminer', repos='http://cran.rstudio.com/')
#install.packages('cmprsk', repos='http://cran.rstudio.com/')
#install.packages('kableExtra', repos='http://cran.rstudio.com/')
#install.packages('arsenal', repos='http://cran.rstudio.com/')
#install.packages('tidyr', repos='http://cran.rstudio.com/')
#install.packages('coin')

In [ ]:
library(dplyr)
library(stringr)
library(forcats)
library(tidyr)
library(ggplot2)
library(survival)
library(ggtext)
library(quantreg)
library(DescTools)
library(survminer)
library(cmprsk)
library(kableExtra)
library(arsenal)
library(tibble)
library(coin)

In [ ]:
options(repr.plot.width=18, repr.plot.height=9)

# Installing the library and connecting to the database using the PICSURE API 

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
session <- picsure::bdc.initializeSession(PICSURE_network_URL, token)
session <- picsure::bdc.setResource(session = session,  resourceName = "AUTH")

# Querying the data

In [ ]:
dictionary_df <- picsure::bdc.searchPicsure(session, 'ORCHID')  
list_variables <- dictionary_df$name

query <- picsure::bdc.newQuery(session) # Initialize a new query
query <- picsure::bdc.addClause(query = query, 
                                keys = list_variables,
                                type = 'ANYOF') 

raw_df <-  picsure::runQuery(query) %>% as_tibble()

In [ ]:
head(raw_df)

In [ ]:
raw_df <- raw_df %>% filter(`\\phs002299\\bl_sex\\` != '')

# Data Management

The raw data contains long variable names, and the following code trims the uniformative part.

In [ ]:
simplified_names <- names(raw_df) %>%
  str_extract('([\\w\\s\\(\\)]+(?=\\\\$))|(Patient\\sID)')
orchid_data <- raw_df[!is.na(simplified_names)]
names(orchid_data) <- simplified_names[!is.na(simplified_names)]
orchid_data <- orchid_data  %>%
  lapply(function(column) {
      if (is.character(column)) {
          as.factor(column) }
      else {
          column
        }
      }
    ) %>%
  as_tibble()

In [ ]:
head(raw_df)

Here we will rename the variables  we will use throughout this notebook.

In [ ]:
outcomes_labels <- c(
  'safe_seizure' = 'Seizure',
  'safe_vtach' = 'Ventricular tachyarrhythmia',
  'safe_ca' = 'Cardiac arrest treated with CPR',
  'safe_astalt' = 'AST or ALT ≥2 times upper limit of normal',
  'safe_hypogly' = 'Symptomatic hypoglycemia',
  'safe_cytopenia' = 'Cytopenia',
  'd_covid3' = 'COVID Outcomes Scale score at day 2',
  'd_covid8' = 'COVID Outcomes Scale score at day 7',
  'd_covid15' = "COVID Outcomes Scale score at day 14, median (IQR)",
  'd_covid29' = 'COVID Outcomes Scale score at day 28',
  'd_mort15' = 'All-cause, all-location death at Day 14',
  'd_mort29' = 'All-cause, all-location death at Day 28',
  'd_ecmo_death' = 'Composite of death or ECMO through 28 d, No./total No. (%)',
  'd_time_to_recovery' = 'Time to recovery in days, median (IQR)',
  'd_hospfreedays' = 'Hospital-free days',
  'd_icufreedays' = 'ICU-free days',
  'd_oxyfreedays' = 'Oxygen-free days',
  'd_vasofreedays' = 'Vasopressor-free days',
  'd_ventfreedays' = 'Ventilator-free days'
)

table1_labels <- c(
    'bl_sex' = 'Sex',
    'bl_age' = 'Age',
    'race_ethnicity' = 'Ethnicity',
    'bl_prehosp' = 'Location at hospitalization',
    'bmi' = 'BMI' ,
    'diabetes_bool' = 'Diabetes',
    'kidney_bool' = 'Chronic Kidney Disease',
    'cad_bool' = 'Coronary Artery Disease',
    'hypertension_bool' = 'Hypertension',
    'copd_bool' = 'Chronic Obstructive Pulmonary Disease',
    'bl_randloc' = 'Location at randomization',
    'covid_ooscale_1' = 'COVID Clinical Scale at Baseline',
    'vit_vasop_1' = 'Vasopressor',
    'd_sofa_gcs' = 'SOFA at baseline',
    'ie_arisymptoms___cou'= 'Cough',
    'ie_arisymptoms___fev'= 'Fever (temperature >37.5 °C)',
    'ie_arisymptoms___sob'= 'Sobbering',
    'ie_arisymptoms___st'= 'Shortness of breath',
    'd_onset_duration'= 'Symptoms onset duration',
    'labs_wbch_1' = 'Lowest white cells blood count at Day 1(/mm3)',
    'labs_wbcl_1' = 'Highest white cells blood count at Day 1(/mm3)',
    'sofa_platl_1' = 'Lowest platelet count at Day 1(k/mm3)',
    'labs_asth_1' = 'Highest AST - Aspartate aminotransferase (units per liter) (Day 1)'
)
tibble("Variable Name" = names(c(outcomes_labels, table1_labels)), 
      "Meaning" = c(outcomes_labels, table1_labels))

Here we define the group of variables used for the analysis

In [ ]:
primary_outcome_name <- "d_covid15"
secondary_outcomes_death <- c("d_mort15",
                               "d_mort29",
                               "d_ecmo_death")
secondary_outcomes_ccs <- c('d_covid3',
                            'd_covid8',
                            'd_covid29')
secondary_outcomes_daysfree <- c(
  'd_time_to_recovery',
  "d_hospfreedays",
  "d_oxyfreedays",
  "d_ventfreedays",
  "d_vasofreedays",
  "d_icufreedays"
)
safety_outcomes <- c(
  'safe_cytopenia',
  'safe_seizure',
  'safe_astalt',
  'safe_hypogly',
  'safe_ca',
  'safe_vtach'
)
chronic_conditions <- c(
    'charl_contis',
    'charl_ulcer',
    'charl_hypertension',
    'charl_cad',
    'charl_fatal',
    'charl_liver',
    'charl_diabetes',
    'charl_kidney',
    'charl_copd'
)
secondary_outcomes_binary <- c(safety_outcomes, secondary_outcomes_death)
ordered_outcome_names <- c(primary_outcome_name, secondary_outcomes_ccs, secondary_outcomes_daysfree)

The code below recode some variables

In [ ]:
orchid_data$rand_trt <- relevel(orchid_data$rand_trt, 'Placebo')

# Recoding cytopenia
orchid_data$safe_cytopenia <- apply(
  orchid_data[, c('safe_neutrop',
                  'safe_lympho',
                  'safe_anemia',
                  'safe_thombo')],
  1,
  function(row) ifelse(any(row == 'Yes', na.rm = TRUE), 'Yes', 'No'))

# Recoding respiratory outcomes
respiratory_symptoms_variables <- c(
    'ie_arisymptoms___cou',
    'ie_arisymptoms___fev',
    'ie_arisymptoms___sob',
    'ie_arisymptoms___st'
)
orchid_data[respiratory_symptoms_variables] <- lapply(orchid_data[respiratory_symptoms_variables], 
                                                      function(variable)
                                                      ifelse(variable == "Checked",
                                                             TRUE,
                                                             ifelse(variable == "Unchecked", 
                                                                    FALSE,
                                                                    NA)
                                                            )
                                                      )

dm_factors <- function(factor) {
  factor <- as.character(factor)
  ifelse(factor == '', NA, ifelse(
    factor %in% c('Yes', '1'), 'Yes', ifelse(
      factor %in% c('No', '0'), 'No', NA
    )
  )) %>% as.factor()
}
orchid_data[secondary_outcomes_binary] <- lapply(
  orchid_data[secondary_outcomes_binary],
  dm_factors) %>%
  as_tibble()


create_ordered_daysfree <- function(variable) {
  factor_levels <- as.character(0:28)
  factor_variable <- factor(as.character(variable), ordered = TRUE, levels = factor_levels)
  return(factor_variable)
}
orchid_data[secondary_outcomes_daysfree] <- lapply(
  orchid_data[secondary_outcomes_daysfree], create_ordered_daysfree) %>%
  as_tibble()


# Transforming COVID Clinical scale into ordered factors
create_ordered_covid_clinical_scale <- function(variable) {
  levels <- c(
    '1, Dead',
    '2, Hospitalized on invasive mechanical ventilation or ECMO',
    '3, Hospitalized on non-invasive ventilation or high flow nasal cannula',
    '4, Hospitalized on supplemental oxygen,',
    '5, Hospitalized not on supplemental oxygen',
    '6, Not hospitalized with limitation in activity (continued symptoms)',
    '7, Not hospitalized without limitation in activity (no symptoms)'
  )
  factor(variable, levels = levels, ordered = TRUE)
}
orchid_data[grep('(^d_covid)|(^covid_ooscale)', names(orchid_data))] <- lapply(
  orchid_data[grep('(^d_covid)|(^covid_ooscale)', names(orchid_data))],
  create_ordered_covid_clinical_scale) %>%
  as_tibble()

# Recoding death
orchid_data$vs_died[orchid_data$vs_died == ''] <- NA
orchid_data$vs_died <- fct_recode(orchid_data$vs_died, '0' = 'No', '1' = 'Yes') %>%
  as.character() %>%
  as.numeric()

# Recoding ethnicity
orchid_data$multi_race <- apply(
  orchid_data[c('bl_race___1', 'bl_race___othcat', 'bl_race___5')],
  1, 
  function(row) ifelse(sum(row == 'Checked', na.rm = TRUE) > 1, TRUE, FALSE))
  
orchid_data$race_ethnicity <- 
  ifelse(orchid_data$bl_ethnic == 'Hispanic or Latino',
         'Hispanic or Latino', 
         ifelse(orchid_data$multi_race == TRUE, 
                'Multirace', 
                ifelse(orchid_data$bl_race___1 == 'Checked', 
                       'Black or African American',
                       ifelse(orchid_data$bl_race___5 == 'Checked', 
                              'White', 
                              ifelse(orchid_data$bl_race___othcat == 'Checked', 
                                     'American Indian or Alaska Native/Asian/Native Hawaiian or Other Pacific Islander',
                                     NA)
                       )
                )
         )
  )

# BMI
orchid_data$bmi <- round(orchid_data$bl_weight / (orchid_data$bl_height / 100)^2, 1)

# Chronics comorbidities
orchid_data$diabetes_bool <- ifelse(orchid_data$charl_diabetes %in%
                                      c('DM with end organ damage present (excludes diet controlled alone)',
                                        'Uncomplicated DM present (no end organ damage present)'),
                                    TRUE, FALSE)
orchid_data$kidney_bool <- ifelse(
    orchid_data$charl_kidney %in% c(
    'Moderate to severe kidney disease present (Cr > 3, ESRD, chart diagnosis of CKD stage 5 (eGFR < 15 mL/min/1.73m²) not on dialysis)',
    'Moderate to severe kidney disease present and patient is dialysis dependent'), 
     TRUE, 
     FALSE)
orchid_data$cad_bool <- ifelse(orchid_data$charl_cad == 'Yes', TRUE, FALSE)
orchid_data$hypertension_bool <- ifelse(orchid_data$charl_hypertension == 'Yes', TRUE, FALSE)
orchid_data$copd_bool <- ifelse(orchid_data$charl_copd == 'Yes', TRUE, FALSE)

In [ ]:
# Defining the general theme for the plots
orchid_gg_theme <-  function(...) {
  theme_bw() +
  theme(legend.background = element_blank(),
        legend.box.background = element_rect(colour = "black"),
        panel.spacing = grid::unit(c(0), "lines"),
        axis.line = element_line(colour = "black"),
        panel.grid.major = element_blank(),
        text = element_text(size=20),
        plot.title = element_text(size = 20, face = 'bold'),
        axis.text.x = element_text(angle=0, hjust=0.5),
        legend.key.height = unit(.8, "cm"),
        panel.border = element_blank(),
        panel.background = element_blank(),
        strip.background = element_rect(colour = "white", fill = "white")) +
    theme(...)
} 

# Population Description

The following code calculate statistics about baseline characteristics of the population according to treatment groups.

In [ ]:
orchid_table1 <- orchid_data
orchid_table1$rand_trt <- relevel(orchid_table1$rand_trt, 'Hydroxychloroquine')

for (label_name in names(table1_labels)) {
    attr(orchid_table1[[label_name]], 'label') <- table1_labels[[label_name]]
}



In [ ]:
demographics_table <- tableby(rand_trt ~ 
                             bl_sex +
                             bl_age +
                             race_ethnicity 
                             , data = orchid_table1)

baseline_table <- tableby(rand_trt ~ 
                             bl_randloc +
                             covid_ooscale_1 +
                             vit_vasop_1 
                             , data = orchid_table1)

comorbidity_table <- tableby(rand_trt ~ 
                             diabetes_bool +
                             kidney_bool +
                             cad_bool +
                             hypertension_bool +
                             copd_bool
                             , data = orchid_table1)

symptoms_table <- tableby(rand_trt ~
                          ie_arisymptoms___cou +
                          ie_arisymptoms___fev +
                          ie_arisymptoms___sob +
                          ie_arisymptoms___st,  
                         data = orchid_table1)

admission_table <- tableby(rand_trt ~ d_sofa_gcs + d_onset_duration,
                              data = orchid_table1
)

table1_df <- merge(demographics_table, baseline_table) %>% 
merge(symptoms_table) %>%
merge(admission_table) %>%
merge(comorbidity_table) %>%
as_tibble() %>%
dplyr::select(all_of(c('label', 'Hydroxychloroquine', 'Placebo'))) %>%
rename('Characteristic' = label) %>%
filter(!Characteristic %in% c('FALSE', 'Range'))


In [ ]:
table1_df[, c('Hydroxychloroquine', 'Placebo')] <- apply(table1_df[, c('Hydroxychloroquine', 'Placebo')],
       c(1, 2), 
       function(elem) {
           elem_bis <- unlist(elem)
           ifelse(elem != '', 
                  unlist(elem_bis) %>%
                  round(1) %>%
                  paste0(collapse = ' (') %>%
                  paste0(')'),
                  elem_bis
                  )
     }
) %>% as_tibble()

In [ ]:
indices_cat <- which(table1_df['Hydroxychloroquine'] == '')
indices_results <- which(table1_df['Hydroxychloroquine'] != '')

kbl(table1_df, caption = "Table 1: Description of the population") %>%
kable_minimal() %>% kable_styling(bootstrap_options = c("striped"), 
                                  full_width = FALSE,
                                  position = "left", 
                                 font_size = 12) %>%
  column_spec(1, width = "20em") %>%
  column_spec(2, width = "10em") %>% 
  column_spec(3, width = "10em") %>% 
add_header_above(c(" ", "No. (%)" = 2), align = 'left') %>%
row_spec(0, bold = TRUE, extra_css = "font-size: medium") %>%
row_spec(indices_cat, underline = FALSE, bold = TRUE, align = 'left') %>%
column_spec(1, background = 'FDEBD0') %>%
kableExtra:::add_indent_html(positions = indices_results) %>%
pack_rows('Demographics', 1, 20, indent = FALSE, label_row_css = "text-align: left; font-style: italic") %>%
pack_rows('COVID-19 Symptoms', 20, 42, indent = FALSE, label_row_css = "text-align: left; font-style: italic") %>%
pack_rows('Comorbidities', 43, 52, indent = FALSE, label_row_css = "text-align: left; font-style: italic") %>%
as.character() %>%
str_replace_all(c("Comorbidities"),
            replacement = function(string) paste0(string, '<hr>')) %>%
str_replace_all(c("Demographics"),
            replacement = function(string) paste0(string, '<hr>')) %>%
str_replace_all(c("COVID-19 Symptoms"),
            replacement = function(string) paste0(string, '<hr>')) %>%
IRdisplay::display_html()

# Outcome Analysis

## Primary Outcome: COVID-19 Ordinal Outcome Scale at Day 14

Primary outcome evolution: evolution of the COVID-19 Ordinal Outcome Scale between day 0 and day 14.

In [ ]:
covid_ooscale_renaming <- c(
    "1, Dead" = "1: Death",
    "2, Hospitalized on invasive mechanical ventilation or ECMO" = "2: Invasive mechanical ventilation or ECMO",
    "3, Hospitalized on non-invasive ventilation or high flow nasal cannula" = "3: Non-invasive ventilation or high flow nasal cannula",
    "4, Hospitalized on supplemental oxygen," = "4: Hospitalized with oxygen",
    "5, Hospitalized not on supplemental oxygen" = "5: Hospitalized without oxygen",
    "6, Not hospitalized with limitation in activity (continued symptoms)" = "6: Discharged, limitation in activity",
    "7, Not hospitalized without limitation in activity (no symptoms)" = "7: Discharged, no limitation in activity"
)

In [ ]:
coos_df <- orchid_data %>%
  dplyr::select(subject_id, rand_trt, starts_with("d_covid")) %>%
  pivot_longer(cols = starts_with("d_covid"),
               names_to = "date",
               values_to = "COOS") %>%
  mutate(COOS =  fct_rev(COOS)) %>%
  mutate(COOS = recode(COOS, !!!covid_ooscale_renaming))


plot_count_COOS <- coos_df %>%
  filter(date %in% c("d_covid15", "d_covid29")) %>%
  mutate(rand_trt = fct_recode(relevel(rand_trt, 'Hydroxychloroquine'),
                               "Hydroxychloroquine\n(n=242)" = "Hydroxychloroquine",
                               "Placebo\n(n=237)" = "Placebo"),
         date = fct_recode(date,
                           "14 d After randomization\n(primary outcome)" = "d_covid15",
                           "28 d After randomization\n(secondary outcome)" = "d_covid29")) %>%
  ggplot() +
  geom_bar(aes(x = rand_trt, fill = COOS), 
           width = 0.4, position="fill", colour = "black") +
  scale_y_continuous(labels = scales::percent,
                     n.breaks = 6,
                     limits = c(0, 1)) +
  facet_grid( ~ date) +
  scale_fill_brewer(palette = "Blues") +
  orchid_gg_theme(plot.title = element_markdown()) +
  labs(title="Clinical Status on the Coronavirus Disease (COVID) Outcomes Scale 14 Days and 28 Days After Randomization",
       fill = "Clinical status (COVID Outcomes Scale category)"
       ) +
  xlab("Treatment group") +
  ylab("Patients with clinical status, %")

plot_data <- ggplot_build(plot_count_COOS)$data
line_data <- plot_data[[1]] %>%
    mutate(x_line = if_else(x == 1, xmax, xmin), 
           date = as.factor(if_else(PANEL == 1, 
                           "14 d After randomization\n(primary outcome)",
                           "28 d After randomization\n(secondary outcome)"
                            ))
          )
         
plot_count_COOS <- plot_count_COOS + 
    geom_line(data = line_data,
              aes(x = x_line, y = ymax, group = fill), 
              linetype = "dashed"
              )
print(plot_count_COOS)

## Primary and secondary outcomes comparison

Below we define a group of functions to analyze the outcomes

In [ ]:
# Functions defined in this cell are being used to analysis the variables formatted as ordered factors
fitted_polr <- function(outcome, covariates, data) {
  formula <- paste(outcome,
                   paste(covariates, collapse = " + "),
                   sep = " ~ ")
  fitted_model <- MASS::polr(formula, Hess = TRUE, dat = data)
  return(fitted_model)
}

dm_polr_results <- function(fitted_model) {
  coef_table <- coef(summary(fitted_model))['rand_trtHydroxychloroquine',]
  OR <- exp(coef(fitted_model))['rand_trtHydroxychloroquine']
  coeff_names <- names(OR)
  ci <- exp(coef_table['Value'] + qnorm(c(0.025, 0.975))*coef_table['Std. Error'])
  p <- pnorm(abs(coef_table["t value"]), lower.tail = FALSE) * 2
  result_model_polyr <- round(c(OR, ci, p), 2)
  names(result_model_polyr) <- c('OR', '2.5%', '97.5%', 'pvalue')
  df_results = as.data.frame(result_model_polyr)
  df_results[['attribute']] = row.names(df_results)
  df_results[['entity']] = 'adjusted'
  df_results[['group']] = 'all'
  df_results[['outcome_type']] = 'Outcomes'
  df_results[['string']] = paste0(result_model_polyr['OR'],
                                  ' (',
                                  result_model_polyr['2.5%'],
                                  ' to ', 
                                  result_model_polyr['97.5%'],
                                  ')'
                                  )
  names(df_results)[names(df_results) == 'result_model_polyr'] = 'value'
  return(df_results)
}


ordered_factor_to_numeric <- function(column) {
  col <- as.character(column)
  matching <- regexpr('^[0-9]+', col)
  as.numeric(
      ifelse(matching != -1, 
            regmatches(col, matching), 
            col)
  )
}


ordered_factors_analysis <- function(ordered_outcome_names, covariates, data) {        
    list_results_outcomes <- list()
    for (outcome_name in ordered_outcome_names) {
      fitted_model <- fitted_polr(outcome_name, covariates, data)
      df_results_polyr <- dm_polr_results(fitted_model)
      df_quartiles <<- get_quartiles(outcome_name, 'rand_trt', data)
      difference <<- df_quartiles[df_quartiles$attribute == '50%' &
                                df_quartiles$group == 'Hydroxychloroquine', 
                                'value'] - 
      df_quartiles[df_quartiles$attribute == '50%' &
                                df_quartiles$group == 'Placebo', 
                                'value']
      df_differences <- ci_quantreg(outcome = outcome_name,
                                    group = 'rand_trt',
                                    difference = difference, 
                                    df = data)
      col_order = c('group', 'entity', 'attribute', 'value', 'outcome_type', "string")
      df_result_outcome <- list(df_results_polyr, df_quartiles, df_differences) %>% 
        lapply(function(df) df[col_order]) %>%
        do.call(rbind, args = .)
      row.names(df_result_outcome) <- NULL
      list_results_outcomes[[outcome_name]] <- df_result_outcome
    }
  return(list_results_outcomes)
}

In [ ]:
# Functions to analyze continuous outcomes
get_quartiles <- function(outcome, group, data) {
  trt_group <- relevel(data[[group]], 'Hydroxychloroquine')
  numeric_var <- ordered_factor_to_numeric(data[[outcome]])
  df_quartiles <- tapply(numeric_var,
         list(trt_group),
         FUN = function(var) {
      trunc(quantile(var, probs = c(0.25, 0.5, 0.75), na.rm = TRUE))          
    }
  ) %>%
    do.call(data.frame, args = .) %>%
    t() %>%
    as.data.frame()
  df_quartiles["string"] = paste0(df_quartiles[["50%"]], " (", df_quartiles[["25%"]], "-", df_quartiles[["75%"]], ")")
  df_quartiles[['entity']] = 'count'
  df_quartiles[['group']] = row.names(df_quartiles)
  df_results <- pivot_longer(df_quartiles,
               cols = c("25%", "50%", "75%"), 
               names_to = 'attribute', 
              values_to = 'value') %>%
    mutate(outcome_type = 'Outcomes')
 
}

ci_quantreg <- function(outcome, group, difference, df) {
  numeric_var <- ordered_factor_to_numeric(df[[outcome]])
  non_nan <- !is.na(numeric_var)
  qr_b <- boot.rq(cbind(1, df[[group]][non_nan]), numeric_var[non_nan], tau = 0.5, R = 10000)
  ci <- t(apply(qr_b$B, 2, quantile, c(0.025,0.975)))[2, ]
  df_results <- data.frame('value' = ci)
  df_results[['entity']] = 'unadjusted'
  df_results[['attribute']] = row.names(df_results)
  df_results[['group']] = 'all'
  df_results[['outcome_type']] = 'Outcomes'
  df_results[['string']] = paste0(difference, ' (', ci[['2.5%']], ' to ',  ci[['97.5%']], ')')
  return(df_results)
}

In [ ]:
# Functions to analyze binary outcomes
fitted_logisticr <- function(outcome, covariates, data) {
    model <- as.formula(paste0(outcome, '~', paste(covariates, collapse = '+')))
    glm(model, data = orchid_data, family = binomial(link='logit'))
}
           
dm_result_logistr <- function(fitted_model, outcome) {
  coef_table <- coef(summary(fitted_model))['rand_trtHydroxychloroquine',]
  OR <- exp(coef(fitted_model))['rand_trtHydroxychloroquine']
  ci <- exp(coef_table['Estimate'] + qnorm(c(0.025, 0.975))*coef_table['Std. Error'])
  results <- round(c(OR, ci), 2)
  df_results = data.frame('value' = results)
  df_results[['attribute']] = c('OR', '2.5%', '97.5%')
  if (any(is.infinite(ci))) df_results[['value']] <- NA 
  df_results[['entity']] = 'adjusted'
  df_results[['group']] = 'all'
  df_results[['outcome_type']] = if (outcome %in% safety_outcomes) 'Safety Outcomes' else 'Outcomes'
  df_results[['string']] = paste0(df_results[df_results$attribute == "OR", "value"],
                                    " (", 
                                    df_results[df_results$attribute == "2.5%", 'value'],
                                    " - ",
                                    df_results[df_results$attribute == "97.5%", 'value'], 
                                   ")")
  return(df_results)
}

get_table_count <- function(data, binary_outcome, safety_outcomes) {
 non_na <- ! (is.na(data[[binary_outcome]]) | data[[binary_outcome]] == '')
 table_count <- table(data[non_na, c('rand_trt', binary_outcome)]) %>%
   as.data.frame() %>%
   pivot_wider(names_from = all_of(binary_outcome), values_from = 'Freq') %>%
   mutate(freq = round(Yes/(Yes + No), 3) * 100) %>%
   pivot_longer(cols = c('No', 'Yes', 'freq'), 
               names_to = 'attribute') %>%
   mutate(entity = 'count') %>%
   rename(group = rand_trt)
   table_count[['outcome_type']] = if (binary_outcome %in% safety_outcomes) 'Safety Outcomes' else 'Outcomes'
   values <- table_count[["value"]]             
   names(values) = table_count[["attribute"]]               
   string = tapply(values, table_count$group,
          function(vector) paste0(vector[["Yes"]], 
                                  " (",
                                  round(vector[["Yes"]]*100/(vector[["Yes"]] + vector[["No"]]), 1), 
                                  ")")
                             )
   df_string = string %>% data.frame()
   names(df_string) = c("string")
   df_string[["group"]] = row.names(df_string)
   inner_join(table_count, df_string, by = "group")         
}

In [ ]:
## Binary outcomes : binomial CI + logistic regression
binary_outcomes_analysis <- function(binary_outcomes,
                                     covariates, 
                                     data) {
    list_results_outcomes <- list()
    for (binary_outcome in binary_outcomes) {
      if (binary_outcome %in% secondary_outcomes_death) {
          model_covariates = covariates
      } else {
          model_covariates = 'rand_trt'
      }
      fitted_model <- fitted_logisticr(outcome = binary_outcome, 
                                        covariates = model_covariates, 
                                        data = data)
      df_logistr <- dm_result_logistr(fitted_model, binary_outcome)
        

     table_count <- get_table_count(data, binary_outcome, safety_outcomes)

      df_differences <- BinomDiffCI(
          x1 = as.numeric(
              table_count[table_count$group == 'Placebo' & 
                          table_count$attribute == 'Yes',
                          'value']
              ), 
          n1 = sum(table_count[table_count$group == 'Placebo' & 
                                       table_count$attribute %in% c('Yes', 'No'),
                                        'value'] ),
          x2 = as.numeric(table_count[table_count$group == 'Hydroxychloroquine' & 
                                       table_count$attribute == 'Yes',
                                        'value']),
          n2 = sum(table_count[table_count$group == 'Hydroxychloroquine' & 
                                       table_count$attribute %in% c('Yes', 'No'),
                                        'value'] ),
                  method = "wald") %>%
        round(3) %>%
        `*`(100) %>% 
        t() %>%
        data.frame(value = .)
      df_differences[["string"]] = paste0(df_differences["est", "value"], 
       " (", 
       df_differences["lwr.ci", "value"], 
       " to ", 
       df_differences["upr.ci", "value"],
       ")")

      df_differences[['entity']] = 'unadjusted'
      df_differences[['attribute']] = row.names(df_differences)
      df_differences[['group']] = 'all'  
      df_differences[['outcome_type']] = if (binary_outcome %in% safety_outcomes) 'Safety Outcomes' else 'Outcomes'
      col_order = c('group', 'entity', 'attribute', 'value', 'outcome_type', "string")
      df_result_outcome <- list(df_logistr, table_count, df_differences) %>% 
        lapply(function(df) df[col_order]) %>%
        do.call(rbind, args = .)
      row.names(df_result_outcome) <- NULL
      list_results_outcomes[[binary_outcome]] <- df_result_outcome
    }
  return(list_results_outcomes)
}

In [ ]:
data_manage_results <- function(list_results_ordered, list_results_binary) {
    outcome_comparison <- do.call(rbind, c(list_results_ordered, list_results_binary, deparse.level = 0))
    outcome_comparison[['outcome']] <- gsub('\\.[0-9]+$', '', row.names(outcome_comparison))
    row.names(outcome_comparison) <- NULL
    outcome_comparison$outcome_type <- factor(outcome_comparison$outcome_type,
                                              levels = c('Outcomes', 'Safety Outcomes'))
    outcome_comparison$outcome <- factor(outcome_comparison$outcome, 
                                         levels = c(safety_outcomes, 
                                                    secondary_outcomes_daysfree, 
                                                    secondary_outcomes_death, 
                                                    secondary_outcomes_ccs, 
                                                    primary_outcome_name)
                                         )
    return(outcome_comparison)
}

data_manage_table <- function(outcome_comparison) {
    outcome_models <- outcome_comparison[c("string", "outcome", "entity")] %>%
    filter(entity %in% c("unadjusted", "adjusted")) %>%
    unique() %>%
    pivot_wider(names_from = "entity", values_from = "string")
    outcome_counts <- outcome_comparison[c("string", "outcome", "group", "entity")] %>%
    filter(entity %in% c("count")) %>%
    unique() %>%
    pivot_wider(names_from = "group", values_from = "string")
    table_results_outcomes <- inner_join(outcome_models, outcome_counts, by = "outcome") %>%
      as.data.frame()
    row.names(table_results_outcomes) <- outcomes_labels[as.character(table_results_outcomes$outcome)]
    table_results_outcomes <- table_results_outcomes[!names(table_results_outcomes) %in% c("outcome", "entity")]
    
    list_renaming_columns <- c(    
        "Hydroxychloroquine" = "Hydroxychloroquine (n = 242)",
        "Placebo" = "Placebo (n = 237)",
        "unadjusted" = "Unadjusted absolute difference (95% CI)",
        "adjusted" = "Adjusted odds ratio or odds ratio (95% CI)"
    )
    table_results_outcomes <- table_results_outcomes[names(list_renaming_columns)]
    names(table_results_outcomes) <- list_renaming_columns[names(table_results_outcomes)]
    return(table_results_outcomes)
}

## Outcomes Analyses

The outcomes will be analyzed in univariate and multivariate analyses. The multivariable model adjust the treatment effect on 5 confoundings: 
- Age at baseline
- Gender
- COVID clinical scale evaluated at baseline (treated as a covariate with 3 levels: 1. invasive or non-invasive ventilation, 2. oxygen supplementation, and 3. no oxygen supplementation)
- SOFA score at baseline
- Time between inclusion and symptoms onset

In [ ]:
# Creation of dummy variables based on the COVID clinical scale at inclusion 
oo_scale1_l3 <- ifelse(as.numeric(orchid_data$covid_ooscale_1) == 3, 
                      TRUE, 
                      FALSE)
oo_scale1_l4 <- ifelse(as.numeric(orchid_data$covid_ooscale_1) == 4, 
                      TRUE, 
                      FALSE)
oo_scale1_l5 <- ifelse(as.numeric(orchid_data$covid_ooscale_1) == 5, 
                      TRUE, 
                      FALSE)

orchid_data$sex_dm <- ifelse(
    orchid_data$bl_sex == "Other/Unknown", 
    NA,
    orchid_data$bl_sex
)

# Set of variables included in the adjusted model
covariates <- c('rand_trt',
                'bl_age',
                'sex_dm',
                'd_sofa_gcs',
                'bl_symptomdt', 
                "oo_scale1_l3",
                "oo_scale1_l4",
                "oo_scale1_l5")

In [ ]:
list_results_ordered <- ordered_factors_analysis(ordered_outcome_names, 
                                                 covariates = covariates, 
                                                 data = orchid_data)

list_results_binary <- binary_outcomes_analysis(c(secondary_outcomes_death, safety_outcomes), 
                                                covariates,
                                                orchid_data)

In [ ]:
outcome_comparison <- data_manage_results(list_results_ordered, list_results_binary)
table_results_outcomes <- data_manage_table(outcome_comparison)

In [ ]:
table_results <- table_results_outcomes %>%
  kable(caption="Effect of Hydroxychloroquine on COVID Outcomes Scale") %>%
  kable_minimal() %>%
  pack_rows('Primary Outcome', 1, 1, indent = FALSE, label_row_css = "text-align: left; font-style: italic") %>%
  pack_rows('Secondary Outcomes', 2, 2, indent = FALSE, label_row_css = "text-align: left; font-style: italic") %>%
  pack_rows('COVID Outcomes Scale score, median (IQR)', 2, 4, indent = TRUE, label_row_css = "text-align: left") %>%
  pack_rows('Support-free days through day 28, median (IQR)', 6, 10, indent = TRUE, label_row_css = "text-align: left") %>%
  pack_rows('All-cause, all-location death, No. (%)', 11, 12, indent = TRUE, label_row_css = "text-align: left") %>%
  pack_rows('Systematically collected safety events, No. (%)', 14, 19, indent = FALSE, label_row_css = "text-align: left; font-style: italic") %>%
  as.character() %>%
str_replace_all(c("Primary Outcome"),
            replacement = function(string) paste0(string, '<hr>')) %>%
str_replace_all(c("Secondary Outcomes"),
            replacement = function(string) paste0(string, '<hr>')) %>%
str_replace_all(c("Systematically collected safety events, No\\. \\(\\%\\)"),
            replacement = function(string) paste0(string, '<hr>')) %>%
str_replace("Placebo \\(n = 237\\)",
            replacement = "Placebo <br>(n = 237)") %>%
str_replace("Hydroxychloroquine \\(n = 242\\)",
            replacement = "Hydroxychloroquine <br>(n = 242)") %>%
str_replace("COVID Outcomes Scale score, median \\(IQR\\) at Day 14",
            replacement = "<b>Outcomes Scale score, median (IQR) at Day 14</b>") %>%
  as.character() %>%
  IRdisplay::display_html()  

In [ ]:
forest_plot <- function(table_results_outcomes, outcomes_labels) {
  outcomes_labels["d_covid15"] <- "**COVID Outcomes Scale score, median (IQR) at Day 14**"
  plt <- table_results_outcomes[table_results_outcomes$entity == 'adjusted' & 
                          ! is.na(table_results_outcomes$value), ] %>%
  pivot_wider(names_from = attribute, values_from = value) %>%
ggplot(aes(x = outcome, y = OR, ymin = `2.5%`, ymax = `97.5%`)) + 
  geom_pointrange() + 
  coord_flip() + 
  facet_grid(outcome_type ~ . ,
             scales = 'free', 
            space = 'free', 
            switch = 'y') +
  geom_hline(yintercept = 1) +
  scale_x_discrete(labels=outcomes_labels) +
  # scale_y_continuous(labels=c(0, 1, 2, 4, 6, 8)) +
  orchid_gg_theme(panel.spacing = unit(2, "lines"), 
                 axis.text.y = element_markdown()) + 
  labs(title = 'Hydroxychloroquine adjusted effects on the outcomes', 
      x = '', 
      y = 'Odds Ratio')
  return(plt)
}

In [ ]:
suppressWarnings(print(forest_plot(outcome_comparison, outcomes_labels)))

## Survival Outcomes

## All-cause survival

Survival has been analyzed through a Kaplan-Meïer curve. The difference in survival curve was tested using a log-rank test. The adjusted treatment effect for all-cause death was estimated using an adjusted hazard regression model.

In [ ]:
CENSOR_TIME <- 28

In [ ]:
orchid_data$death_within_28 <- ifelse(
    orchid_data$vs_deathdt <= 28,
    1, 
    0
) %>% replace_na(0)

orchid_data$death_fu <- orchid_data$d_lastalivedt + 1
no_fu_individuals <- apply(orchid_data[c('vs_alivedt', 'vs_deathdt')],
                                      1,
                                      function(x) sum(is.na(x)) == 2)

survival_data <- orchid_data[!no_fu_individuals, ]

survival_data$fu_death_28 <- pmin(28, 
                            survival_data$vs_alivedt,
                            survival_data$vs_deathdt,
                           na.rm = TRUE)

coxph_death <- coxph(Surv(time = survival_data$fu_death_28,
                          event = survival_data$death_within_28) ~
                       rand_trt + bl_age + bl_sex + covid_ooscale_1 + d_sofa_gcs_s + bl_symptomdt,
                     data = survival_data)


survfit_death <- survfit(Surv(time = survival_data$fu_death_28,
                              event = survival_data$death_within_28) ~ rand_trt,
                         data = survival_data)

In [ ]:
death_plot <- ggsurvplot(survfit_death,
           xlim = c(1, 28),
           size = 1.3,                 
           palette = c("#275057", "#F4890B"),
           conf.int = FALSE,          
           pval = TRUE,              
           risk.table = FALSE,        
           risk.table.col = "strata",
           legend.labs =c("Placebo", "Hydroxychloroquine"),
           risk.table.height = 0.25,
           ggtheme = theme_bw() +  theme(
        text = element_text(size=20),
        axis.text.x = element_text(angle=0, hjust=0.5)),
           combine = TRUE,
           keep_data = TRUE,
           type="cuminc",
           title = "Survival Through 28 Days Following Randomization",
)

death_plot

### Hospital discharge 

Time to hospital-discharge between treatment groups has been analyzed using a Fin and Gray Model, treating death as a competing risk. A patient was considered discharged at first hospital discharge, regardless of eventual re-hospitalization or death occurring afterwards.

Adjusted treatment effect on time to hospital-discharge was estimated using an adjusted hazard regression model.

In [ ]:
discharge_data <- orchid_data

discharge_data$discharge_status <- with(discharge_data, 
                        ifelse(!is.na(vs_deathdt) & !is.na(iho_dischdt), 
                               ifelse(vs_deathdt <= iho_dischdt, 
                                      "dead",
                                      "discharged"),
                               ifelse(!is.na(vs_deathdt),
                                      "dead",
                                      ifelse(!is.na(iho_dischdt),
                                             "discharged",
                                             "censored")
                                     )
                               )
) %>% factor(levels = c("censored", "discharged", "dead"))
discharge_data$discharge_dt <- with(discharge_data, 
                                    ifelse(                                        
                                        discharge_status == "discharged", 
                                        iho_dischdt, 
                                        ifelse(discharge_status == "dead", 
                                              vs_deathdt, 
                                              ifelse(discharge_status == "censored", 
                                                    vs_alivedt, 
                                                    NA)
                                              )
                                        )
                                    ) %>%
ifelse(. <= CENSOR_TIME, ., CENSOR_TIME)

In [ ]:
discharge_fit <- cmprsk::cuminc(
    ftime = discharge_data$discharge_dt, 
    fstatus = as.numeric(discharge_data$discharge_status), 
    cencode = 1, 
    group = as.numeric(discharge_data$rand_trt)
)
pvalue_gray <- round(discharge_fit[["Tests"]][1, "pv"], 2)

In [ ]:
survfit_discharge <- survfit(Surv(
    discharge_data$discharge_dt, 
    ifelse(discharge_data$discharge_status == "discharged", 1, 0), 
    ) ~ rand_trt, 
    data = discharge_data[c("discharge_dt", "discharge_status", "rand_trt")]
                            )

ggsurvplot(
  survfit_discharge, 
  xlim = c(1, 28),
  size = 1.3,
  risk.table = FALSE,        
  palette = c("#F4890B", "#275057"),
  conf.int = FALSE,          
  pval = pvalue_gray,              
  risk.table.col = "strata",
  legend.labs = c("Hydroxychloroquine", "Placebo"),
  risk.table.height = 0.25,
  ggtheme = theme_bw() +  theme(
  text = element_text(size=20),
  axis.text.x = element_text(angle=0, hjust=0.5)),
  combine = TRUE,
  keep_data = TRUE,
  linetype = 'strata',
  fun = 'event',
  type = "cuminc",
  title = "Hospital Discharge Through 28 Days Following Randomization"
)

# Sensitivity Analyses

This notebook can also be useful to kick-start new analyses. The heavy lifting data management has already been done and allows the user to begin expanding on the original work quickly. As an example, we are going to focus on the lab trajectories in the different treatment groups, and how it is related to the use of premedication by azithromycin.

## Time to recovery associated with premedications

In [ ]:
premedications <- c(
        "prmed_chlor",
        "prmed_hydroxychlor",
        "prmed_remdesivir",
        "prmed_lopinavir",
        "prmed_corticoster",
        "prmed_tocilizumab",
        "prmed_sarilumab",
        "prmed_interferon",
        "prmed_azithro"
    )

In [ ]:
coos_df <- orchid_data %>%
  dplyr::select(subject_id, covid_ooscale_1, prmed_azithro, starts_with("d_covid")) %>%
  pivot_longer(cols = c(starts_with("d_covid"), covid_ooscale_1),
               names_to = "date",
               values_to = "COOS") %>%
  mutate(COOS =  fct_rev(COOS)) %>%
  mutate(COOS = recode(COOS, !!!covid_ooscale_renaming))

In [ ]:
table_outcomes_count <- coos_df %>%
  filter(date %in% c("covid_ooscale_1", "d_covid15", "d_covid29")) %>%
  mutate(prmed_azithro = as.factor(prmed_azithro)) %>%
  mutate(prmed_azithro = fct_recode(relevel(prmed_azithro, 'Yes'),
                               "Yes\n(n=150)" = "Yes",
                               "No\n(n=329)" = "No"),
         date = fct_recode(date,
                           "At randomization" = "covid_ooscale_1", 
                           "14 d After randomization\n(primary outcome)" = "d_covid15",
                           "28 d After randomization\n(secondary outcome)" = "d_covid29"))
 

In [ ]:
table_count_azithro <- table_outcomes_count %>%
  group_by(prmed_azithro, COOS, date) %>%
  summarize(count = n()) %>%
  group_by(prmed_azithro, date) %>%
  mutate(percentage = round(count/sum(count) * 100, 1)) %>%
  mutate(string = glue::glue("{count} ({percentage}%)", count = count, percentage = percentage)) %>% 
  pivot_wider(id_cols = c(COOS),
              names_from = c(date, prmed_azithro), 
              values_from = string) %>%
  arrange(desc(COOS))
table_count_azithro <- table_count_azithro %>%
  mutate(across(colnames(table_count_azithro)[-1], replace_na, "0 (0%)")) %>%
  `[`(c(1, 4, 7, 2, 5, 3, 6))
table_count_azithro

In [ ]:
names(table_count_azithro) <-  c("COOS", 
         "Yes (n=150)",
         "No (n=329)",
         "Yes (n=150)",
         "No (n=329)",
         "Yes (n=150)",
         "No (n=329)")
table_count_azithro %>%
kbl() %>%
kable_minimal() %>% 
kable_styling(bootstrap_options = c("striped"), 
                                full_width = FALSE,
                                 position = "left", 
                                 font_size = 12) %>%
  column_spec(1, width = "20em") %>%
  column_spec(2, width = "10em") %>% 
  column_spec(3, width = "10em") %>% 
add_header_above(c(" " = 1, "At randomization" = 2, 
                  "14 d After randomization" = 2, 
                  "28 d After randomization" = 2),
                 align = 'left') %>%
as.character() %>%
IRdisplay::display_html()

In [ ]:
plot_count_COOS <- table_outcomes_count %>%
  ggplot() +
  geom_bar(aes(x = prmed_azithro, fill = COOS), 
           width = 0.4, position="fill", colour = "black") +
  scale_y_continuous(labels = scales::percent,
                     n.breaks = 6,
                     limits = c(0, 1)) +
  facet_grid( ~ date) +
  scale_fill_brewer(palette = "Oranges") +
  orchid_gg_theme(plot.title = element_markdown()) +
  labs(title="Clinical Status on the Coronavirus Disease (COVID) Outcomes Scale 14 Days and 28 Days After Randomization",
       fill = "Clinical status (COVID Outcomes Scale category)"
       ) +
  xlab("Premedication by azithromycin") +
  ylab("Patients with clinical status, %")

plot_data <- ggplot_build(plot_count_COOS)$data
line_data <- plot_data[[1]] %>%
    mutate(x_line = if_else(x == 1, xmax, xmin), 
           date = as.factor(if_else(PANEL == 1, 
                                    "At randomization",
                                    ifelse(PANEL == 2,
                                           "14 d After randomization\n(primary outcome)",
                                           "28 d After randomization\n(secondary outcome)"
                                           )
                            ))
          )
         
plot_count_COOS <- plot_count_COOS + 
    geom_line(data = line_data,
              aes(x = x_line, y = ymax, group = fill), 
              linetype = "dashed"
              )
print(plot_count_COOS)

This plot represents the patients' clinical status according to day 0, 14, and 28 from randomization. It shows that, overall, patients who received premedication with azithromycin had more severe clinical baseline status. This translated into more severe outcomes at the different time points (day 14 and 28 from randomization).

## Premedications and labs trajectories

In [ ]:
labs_names <- list(
    labs_wbch = "Highest white blood cell count (/mm3)", 
    labs_wbcl = "Lowest white blood cell count (/mm3)", 
    labs_hemol = "Lowest hemoglobin (g/dl)", 
    labs_plateletl = "Lowest platelet count (/mm3; in thousands)", 
    labs_sodiumh = "Highest sodium (mEq/L)", 
    labs_sodiuml = "Lowest sodium (mEq/L)", 
    labs_potassiumh = "Highest potassium (mEq/L)", 
    labs_potassiuml = "Lowest potassium (mEq/L)", 
    labs_chlorideh = "Highest chloride (mEq/L)", 
    labs_bicarbonateh = "Highest bicarbonate (mEq/L)", 
    labs_bicarbonatel = "Lowest bicarbonate (mEq/L)", 
    labs_bunh = "Highest BUN - Blood Urea Nitrogen (mg/dL)", 
    labs_creath = "Highest creatinine (mg/dL)", 
    labs_troph = "Highest troponin (mg/dL)", 
    labs_asth = "Highest AST - Aspartate aminotransferase (units per liter)", 
    labs_alth = "Highest ALT - Alanine aminotransferase (units per liter)", 
    labs_alph = "Highest ALP - Alkaline Phosphatase (IU/L)", 
    labs_totbilih = "Highest total bilirubin (mg/dL)", 
    labs_albuminl = "Lowest albumin (g/dL)", 
    labs_ptth = "Highest PTT - Partial Thromboplastin Time (seconds)", 
    labs_inrh = "Highest INR - International Normalized R"
)

In [ ]:
list_names = list()
for (labs in names(labs_names)) {
    list_names[[labs]] <- grep(labs, names(orchid_data), value = TRUE)
}
orchid_labs_names <- do.call(c, list_names) %>% unname()

In [ ]:
labs_long <- orchid_data[c("Patient ID", "rand_trt", "prmed_azithro", orchid_labs_names)] %>%
    pivot_longer(cols = all_of(orchid_labs_names), 
                 names_sep = "_(?=[0-9]$)", 
                 names_to = c("labs", "day"),
                 values_to = "values") %>%
    mutate(day = as.numeric(as.character(day)))

In [ ]:
summarized_labs <- labs_long %>% 
    group_by(rand_trt, prmed_azithro, labs, day) %>%
    summarise(mean = mean(values, na.rm = TRUE), 
              count = n(), 
              sd = sd(values, na.rm = TRUE)) %>%
   ungroup()

In [ ]:
summarized_labs <- labs_long %>% 
    group_by(rand_trt, prmed_azithro, labs, day) %>%
    summarise(mean = mean(values, na.rm = TRUE), 
              count = n(), 
              sd = sd(values, na.rm = TRUE)) %>%
   ungroup()

In [ ]:
labs_long %>%
group_by(prmed_azithro, labs, day) %>%
summarise(median = median(values, na.rm = TRUE),
          mean = mean(values, na.rm = TRUE),
          sd = sd(values, na.rm = TRUE), 
          n_obs = n()) %>% 
mutate(labs = recode(labs, !!!labs_names), 
       prmed_azithro = relevel(prmed_azithro, ref = "Yes")) %>% {
ggplot(., aes(x = day, 
           y = mean, 
           color = prmed_azithro, 
           shape = prmed_azithro)) +
geom_point(aes(size=n_obs)) +
geom_line() + 
# geom_errorbar(aes(ymin = pmax(0, mean - sd), ymax = mean + sd), width = 0.2) +
scale_size_binned() +
scale_color_brewer(palette = "Set1") +
facet_wrap("labs", scale = "free_y", labeller = label_wrap_gen(width=40)) + 
orchid_gg_theme() +
    theme(strip.text.x = element_text(size=11)) +
labs(title = "Labs trajectory from Day 1 to 5, depending on premedication", 
     subtitle = "Premedication by azythromycin", 
     x = "Days from inclusion", 
     y = "Mean concentration", 
     colour = "Azithromycin",
     shape = "Azithromycin",
     size = "Observation count")
}


Interpretation of these graphs: 

- The most significant differences occur in the average level of troponin: patients that had been prescribed Azithromycin had lower levels of troponin on average.

In [ ]:
labs_long[labs_long$labs == "labs_asth", ] %>%
mutate(prmed_azithro = relevel(prmed_azithro, ref = "Yes")) %>%
   ggplot(aes(x=day, y=values, colour=prmed_azithro)) +
   geom_jitter(size=4) +
   scale_colour_brewer(palette = "Set1") +
   orchid_gg_theme() +
labs(title = paste0(labs_names[["labs_asth"]], " concentation"), 
     subtitle = "Premedication by azythromycin", 
     x = "Days from inclusion", 
     y = labs_names[["labs_troph"]], 
     colour = "Azithromycin")
   

In [ ]:
labs_long[labs_long$labs == "labs_alth", ] %>%
mutate(prmed_azithro = relevel(prmed_azithro, ref = "Yes")) %>%
   ggplot(aes(x=day, y=values, colour=prmed_azithro)) +
   geom_jitter(size=4) +
   scale_colour_brewer(palette = "Set1") +
   orchid_gg_theme() +
labs(title = paste0(labs_names[["labs_alth"]], " concentation"), 
     subtitle = "Premedication by azythromycin", 
     x = "Days from inclusion", 
     y = labs_names[["labs_troph"]], 
     colour = "Azithromycin")
   

In [ ]:
labs_long[labs_long$labs == "labs_alph", ] %>%
mutate(prmed_azithro = relevel(prmed_azithro, ref = "Yes")) %>%
   ggplot(aes(x=day, y=values, colour=prmed_azithro)) +
   geom_jitter(size=4) +
   scale_colour_brewer(palette = "Set1") +
   orchid_gg_theme() +
labs(title = paste0(labs_names[["labs_alph"]], " concentation"), 
     subtitle = "Premedication by azythromycin", 
     x = "Days from inclusion", 
     y = labs_names[["labs_troph"]], 
     colour = "Azithromycin")
   

In [ ]:
labs_long[labs_long$labs == "labs_troph", ] %>%
mutate(prmed_azithro = relevel(prmed_azithro, ref = "Yes")) %>%
   ggplot(aes(x=day, y=values, colour=prmed_azithro)) +
   geom_jitter(size=4) +
   scale_colour_brewer(palette = "Set1") +
   orchid_gg_theme() +
labs(title = paste0(labs_names[["labs_troph"]], " concentation"), 
     subtitle = "Premedication by azythromycin", 
     x = "Days from inclusion", 
     y = labs_names[["labs_troph"]], 
     colour = "Azithromycin")
   

In [ ]:
labs_long[grepl("labs_totbilih", labs_long$labs), ] %>%
mutate(prmed_azithro = relevel(prmed_azithro, ref = "Yes")) %>%
   ggplot(aes(x=day, y=values, colour=prmed_azithro)) +
   geom_jitter(size=4) +
   scale_colour_brewer(palette = "Set1") +
   orchid_gg_theme() +
labs(title = paste0(labs_names[["labs_totbilih"]], " concentation"), 
     subtitle = "Premedication by azythromycin", 
     x = "Days from inclusion", 
     y = labs_names[["labs_totbilih"]], 
     colour = "Azithromycin")
   